In [11]:
# imports for ctgene
import pandas as pd
import numpy as np

# imports for plotting
import toyplot
import toyplot.png

In [12]:
# ctgene function
def ctgene(df, genelist):
    temp = dict.fromkeys(genelist)
    testdigits = ['0','1','2','3','4','5','6','7','8','9']
    
    for key in temp.keys():
        temp[key] = dict.fromkeys(set(df['Shortname']), 0)   
        
    for index, row in df.iterrows():
        for gene in genelist:
            gene_higherdigits = [gene.lower() + i for i in testdigits]
            not_a_digit_error = True
            for i in gene_higherdigits:
                    if i in row['Gene Name'].lower():
                        not_a_digit_error = False
            if gene.lower() in row['Gene Name'].lower() and not_a_digit_error:
                temp[gene][row['Shortname']] += 1
    countdf = pd.DataFrame(temp) #easy to read format but bad for plotting
    countdf.reset_index(inplace=True)
    countdf.rename(columns={'index':'Shortname'}, inplace=True)
    return countdf

In [13]:
def get_MAGids(df):
    return [i.split('SB-')[1] for i in df['Genome Name']]

# renames columns in output dfs from JGI IMG so that they match
# adds MAG Id column ('MAG ##')


def prep_IMG_blast(taxondf, blastdf):
    taxondf = taxondf.rename(columns={'Genome Name / Sample Name' : 'Genome Name', \
                    'IMG Genome ID ': 'Genome Id'})
    taxondf['MAG Id'] = get_MAGids(taxondf)
    blastdf['MAG Id'] = get_MAGids(blastdf)

    shortnames = [''] * len(blastdf['Genome Name'])
    
    for index, row in blastdf.iterrows():
        MAGclass = taxondf[taxondf['Genome Id'] == row['Genome Id']]['Class'].tolist()[0]
        shortnames[index] = MAGclass + ' ' + row['MAG Id']
    
    blastdf['Shortname'] = shortnames 
    
    neatdf = pd.concat([blastdf['Gene Name'], blastdf['Genome Name'], \
                        blastdf['Genome Id'], blastdf['MAG Id'], blastdf['Shortname']], axis=1) 
    
    return (taxondf, blastdf, neatdf)
# rename 'Genome Name / Sample Name' column to match rawdf's 'Genome Name' column

In [9]:
blastdf = pd.read_csv('Data/bacteroidetes-MAGs-blast-bfragilis-conserved-riboprots.csv')
taxondf = pd.read_csv('Data/bacteroidetes-MAGs-taxontable.csv')
taxondf, blastdf, df = prep_IMG_blast(taxondf, blastdf)

In [10]:
genelist = pd.read_csv('Data/conserved-riboprots-list.csv', header=None)[0].tolist()

In [11]:
countsdf = ctgene(df, genelist)
counts = countsdf.to_numpy()
countsmatrix = np.asmatrix(counts[:,1:])
rowlabels = np.append([''], counts[:,0])
collabels = np.append([''],countsdf.columns.tolist()[1:])

In [403]:
canvas = toyplot.Canvas(width=1500, height=600, style={'background-color': 'white'})
table = canvas.table(
    rows=counts.shape[0] + 1,
    columns=counts.shape[1],
    margin=0,
    bounds=(20, 1480, 20, 580),
)

for ridx in range(counts.shape[0]):
    for cidx in range(1,counts.shape[1]):
        cell = table.cells.cell[ridx+1, cidx]
        if counts[ridx, cidx] == 1:
            cell.style = {
                "fill": toyplot.color.css("gray"),
            }
        elif counts[ridx, cidx] > 1:
            cell.style = {
                'fill': toyplot.color.css("black")
            }
            
table.body.gaps.columns[:] = 3
table.body.gaps.rows[:] = 3
table.cells.column[0].data = rowlabels
table.cells.column[0].width = 140
table.cells.column[0].align = 'right'
table.cells.row[0].data = collabels

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1500.0px" height="600.0px" viewBox="0 0 1500.0 600.0" preserveAspectRatio="xMidYMid meet" style="background-color:white;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb4989bf12d87407e9a9d5cf125a3ece2"> S2 S3 S4 S5 S6 S7 S8 S9 S10 S11 S12 S13 S14 S15 S16 S17 S18 S19 S20 L1 L2 L3 L4 L5 L6 L9 L10 L11 L12 L13 L14 L15 L16 L17 L18 L19 L20 L22 L23 L24 L27 L28 L29 L31 L32 L33 L34 L35 L36 Cytophagia MAG 2 Bacteroidia MAG 84 Flavobacteriia MAG 42 Cytophagia MAG 4 Cytophagia MAG 6 Saprospiria MAG 46 Cytophagia MAG 1 Flavobacteriia MAG 3 Cytophagia MAG 69 Bacteroidia MAG 14 Saprospiria MAG 55 Cytophagia MAG 81 Cytophagia MAG 83 Flavobacteriia MAG 15 Cytophagia MAG 56 Cytophagia MAG 5 Saprospiria MAG 62 Cytophagia MAG 45 Saprospiria MAG 11 Flavobacteriia MAG 39

In [404]:
toyplot.png.render(canvas, 'riboprots-presence-absence.png')